In [ ]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display, clear_output
import openai

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Prompt user for company name and URL
company_name = input("Enter the company name: ")
url = input("Enter the company URL: ")

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
# multi-shot prompt
link_system_prompt = "You are provided with a list of links found on a webpage. \You are able to decide which of the links would be most relevant to include in a brochure about the company, \such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
    EXAMPLE 1:
    {
        "links": [
            {"type": "about page", "url": "https://full.url/goes/here/about"},
            {"type": "careers page", "url": "https://another.full.url/careers"}
        ]
    }
    EXAMPLE 2:
    {
        "links": [
            {"type": "company blog", "url": "https://blog.example.com"},
            {"type": "our story", "url": "https://example.com/our-story"}
        ]
    }
    """

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \ Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt


def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)

    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

# set format to json_object
system_prompt = (
    "You are an assistant that analyzes the contents of several relevant pages from a company website "
    "and creates a short tempered, irritated, disappointed in the world type of brochure about the company for prospective customers, investors, and recruits. "
    "Respond in markdown. Include details of company culture, customers, and careers/jobs if you have the information. Add emoticons where ever possible.\n\n"

    "Please structure the brochure using the following sections:\n"
    "1. **Introduction**: A brief overview of the company.\n"
    "2. **Company Culture**: Emphasize fun, atmosphere, and any unique cultural elements.\n"
    "3. **Customers**: Mention notable customers or industries.\n"
    "4. **Careers/Jobs**: Highlight career opportunities.\n"
    "5. **Conclusion**: Wrap up with a final lighthearted message.\n"
    "6. Finish the brochure with a very sarcastic and pun-intended mission statement.\n"
)

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]
    return user_prompt

def stream_brochure():
    global brochure_text  # Access the global variable
    brochure_text = ""    # Initialize
    
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream: 
        content = chunk.choices[0].delta.content or ''
        response += content
        brochure_text += content  # Accumulate the text
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

def user_translate_brochure(lang):
    # Clear previous output
    clear_output(wait=True)
    
    # Stream #2: translate accumulated text
    translation_stream = openai.chat.completions.create(  # Changed from ChatCompletion
        model=MODEL,
        messages=[
            {"role": "user", "content": f"Translate the following to {lang}:\n\n{brochure_text}"}
        ],
        stream=True
    )
    
    # Setup display for streaming translation
    display_handle = display(Markdown(""), display_id=True)
    translated_text = ""
    
    for chunk in translation_stream:
        content = chunk.choices[0].delta.content or ""
        if content:
            translated_text += content
            update_display(Markdown(translated_text), display_id=display_handle.display_id)

# stream the brochure in english
stream_brochure()

# prompt user for language choice
language_choice = input("Enter the language to translate the brochure into (e.g., 'French'): ")

# translate the brochure and stream the translation
user_translate_brochure(language_choice)